In [ ]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
import pprint

In [ ]:
CONSUMER_KEY = '9ll3e9DHy2TJjKoyi0ML3FLkV'
CONSUMER_SECRET = 'skaKNfFxEnfC4sQPMI4dHKtDaTrTdvJ2jVMqKoZBS8z1B3Nxs6'
ACCESS_TOKEN = '80123742-xYyozhxQfyr9bDhf7LODPywKjYhWKOqnrElgjWZug'
ACCESS_TOKEN_SECRET = 'WQ0gsgWuFIFKsDh4h45Cf9TKCo8hyPQTpvRS8LT0SWfFW'

# FPP_KEY = ''
# FPP_SECRET = ''

MONGO_HOST = 'mongodb://localhost:27017/'

In [ ]:
class StreamListener(tweepy.StreamListener):

    def on_connect(self):
        # ao conectar
        print("Conectado a streaming API.")

    def on_error(self, status_code):

        print('Erro: ' + repr(status_code))
        return False

    def on_data(self, data):

        try:
            # especificando o caminho do servidor
            client = MongoClient(MONGO_HOST)

            # explicitando o banco
            db = client.LiscaDoido

           # carregando os dados recebidos e transformando em json
            datajson = json.loads(data)

            created_at = datajson['created_at']

            print("Tweet collected at " + str(created_at))
            print(datajson['text'])

            # informando qual a collection a inserir o documento
            # nesse caso se não houver uma colection com o respectivo nome, ela será criada
            # armazenando nna collection 'streamingAPI'
            db.tweets.insert_one(datajson)
        except Exception as e:
            print(e)

In [ ]:
# fazendo a autenticação OAuth
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
# instanciando a classe que foi criada para gerenciar a conexão
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))

In [ ]:
# criando stream
streamer = tweepy.Stream(auth=auth, listener=listener)

In [ ]:
# print('rastreando por coordenadas')
# streamer.filter(locations=[SWlongitude, SWLatitude, NElongitude, NELatitude])
# streamer.filter(locations=[-55.041526, -2.729070, -54.455144, -2.361513])

In [ ]:
print('rastreando por palavra chave')
# WORDS = ['#bigdata', '#AI', '#datascience', '#machinelearning', '#ml', '#iot']
WORDS = ['Lisca']

In [ ]:
# iniciando o stream
streamer.filter(track=WORDS)